In [2]:
import requests
from bs4 import BeautifulSoup as bs
# import pandas as pd
import json
import tqdm
from os import mkdir
from os.path import join
from shutil import rmtree

In [3]:
basic_url = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=detailed"

typical_1 = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=detailed&page=1"

headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

In [11]:
META_PREFICS = "https://www.metacritic.com"
USER_POSTFICS = "/user-reviews"
CRITIC_POSTFICS = "/critic-reviews"

In [4]:
SAVING_PATH =""
answ = []
def parse_game_page(URL:str,
                    page_count: int,
                    local_save_path:str,
                    json_save:bool = False):
    """
    Gets information about all games on a page (name, comment ,score ,date) and
    or saves it in json format in SAVING_PATH catalog, or returns it as dict.
    :param URL: str - page url
    :param page_count: int - counter to give proper names for json files
    :param name: str - name of the game
    :param json_save: bool - flag that points out to save in json or return as dict
    :return: None or dict full of info (keys: name, comment ,score ,date)
    """

    r = requests.get(URL, headers=headers)

    soup = bs(r.text, "html.parser")
    games_rewievs = soup.find_all(class_="review_content")

    rewiev_count = 0
    for rewiev in games_rewievs:

        # different structure means, that is not user review
        if rewiev.find(class_="review_body").span is None:
            continue

        text = rewiev.find(class_="review_body").span.text
        date = rewiev.find(class_="date").text
        score = rewiev.find(class_="review_grade").div.text

        if json_save:
            with open( local_save_path +f"/page{page_count}_comment{rewiev_count}.json", 'w') as f:
                json.dump({"text of comment": text, "score": score,
                           "date": date}, f)
        else:
             return {"text of comment": text, "score": score,
                     "date": date}

        rewiev_count += 1


In [52]:
def parse_main_page(URL, dict):
    r = requests.get(URL, headers=headers)
    soup = bs(r.text, "html.parser")
    titels = soup.find_all(class_="clamp-summary-wrap")
    for title in titels:
        name = title.find("a", class_="title").h3.text
        ref = META_PREFICS + title.find("a", class_="title")['href'] + USER_POSRFICS
        dict[name] = ref
    return dict

In [92]:
with open("/Users/fedor/Desktop/meta_project/data/metacritic/references.json", "r") as read_file:
    ref_dict = json.load(read_file)

In [93]:
def scan_for_pages(dict, key):
    r = requests.get( dict[key], headers=headers)
    soup = bs(r.text, "html.parser")

    inner_couner = 0
    inner_couner = soup.find(class_ = "page last_page")
    if inner_couner is None:
        first_url = dict[key]
        dict[key] = [first_url]
        return dict

    inner_couner =  inner_couner.a.text

    first_url = dict[key]
    dict[key] = [first_url]
    for i in range(1,int(inner_couner)):
        dict[key].append(first_url+"?page="+str(i))
        dict[key] = dict[key]
    return dict


In [3]:
with open("/Users/fedor/Desktop/meta_project/data/metacritic/all_references.json", "r") as f:
    ref_dict = json.load(f)

In [25]:
ref_dict['BioShock']

['https://www.metacritic.com/game/playstation-3/bioshock/user-reviews',
 'https://www.metacritic.com/game/playstation-3/bioshock/user-reviews?page=1']

In [23]:
def final_parse(dict):
    for game_name in tqdm.tqdm(list(dict.keys())[:20]):
        mkdir("/Users/fedor/Desktop/meta_project/data/metacritic/user_review/"+game_name)
        page_count = 0
        for game_link in dict[game_name]:
            parse_game_page(game_link,
                            page_count
                            , "/Users/fedor/Desktop/meta_project/data/metacritic/user_review/"+game_name
                            , True)
            page_count += 1

In [ ]:
def scan_for_pages(dict, key):
    r = requests.get( dict[key], headers=headers)
    soup = bs(r.text, "html.parser")

    inner_couner = 0
    inner_couner = soup.find(class_ = "page last_page")
    if inner_couner is None:
        first_url = dict[key]
        dict[key] = [first_url]
        return dict

    inner_couner =  inner_couner.a.text

    first_url = dict[key]
    dict[key] = [first_url]
    for i in range(1,int(inner_couner)):
        dict[key].append(first_url+"?page="+str(i))
    return dict

In [76]:
def scan_for_pages_by_url(URL):
    r = requests.get( URL, headers=headers)
    soup = bs(r.text, "html.parser")

    inner_couner = 0
    inner_couner = soup.find(class_ = "page last_page")
    if inner_couner is None:
        return [URL]

    inner_couner =  inner_couner.a.text

    answ = [URL]
    for i in range(1,int(inner_couner)):
        answ.append(URL+"?page="+str(i))
    return answ

In [91]:
final_ref =
for game_name in tqdm.tqdm(list(ref_dict.keys())[3231+1840:6463]):
    final_ref[game_name] = {}
    first_link = ref_dict[game_name][0]
    this_platform = first_link.split("/")[4]
    final_ref[game_name][this_platform] = ref_dict[game_name]

    r = requests.get( first_link, headers=headers)
    soup = bs(r.text, "html.parser")

    platform_det = soup.find(class_ = "summary_detail product_platforms")
    if platform_det is None:
        continue

    platforms = platform_det.find(class_ = "data").find_all("a")
    for p in platforms:
        final_ref[game_name][p.text] = scan_for_pages_by_url(META_PREFICS + "/"+p["href"]+USER_POSRFICS)


100%|██████████| 1392/1392 [1:02:49<00:00,  2.71s/it]


In [99]:
for game_name in tqdm.tqdm(list(ref_dict.keys())[6463+843+2489+576:]):
    final_ref[game_name] = {}
    first_link = ref_dict[game_name][0]
    this_platform = first_link.split("/")[4]
    final_ref[game_name][this_platform] = ref_dict[game_name]

    r = requests.get( first_link, headers=headers)
    soup = bs(r.text, "html.parser")

    platform_det = soup.find(class_ = "summary_detail product_platforms")
    if platform_det is None:
        continue

    platforms = platform_det.find(class_ = "data").find_all("a")
    for p in platforms:
        final_ref[game_name][p.text] = scan_for_pages_by_url(META_PREFICS + "/"+p["href"]+USER_POSRFICS)

100%|██████████| 2556/2556 [1:24:38<00:00,  1.99s/it]


In [9]:
actual_start_pos = 2652+51
loc_path = "/Users/fedor/Desktop/meta_project/data/metacritic/user_review/"
for game_name in tqdm.tqdm(list(final_ref.keys())[actual_start_pos : ]):
    safe_game_name = game_name.replace("/","_")

    if safe_game_name != game_name:
        games_name_changed.append(game_name)

    try:
            mkdir( loc_path + safe_game_name )
    except FileExistsError:
        rmtree(join(loc_path + safe_game_name))
        mkdir(loc_path + safe_game_name)


    for platform in final_ref[game_name]:
        if platform == "iPhone/iPad":
            safe_platform = "iPhone_iPad"
        else:
            safe_platform = platform

        mkdir(loc_path+safe_game_name+"/"+safe_platform)
        page_count = 0

        for game_link in final_ref[game_name][platform]:
            parse_game_page(game_link
                            ,page_count
                            ,loc_path + safe_game_name + "/" + safe_platform
                            , True)
            page_count += 1

100%|██████████| 10224/10224 [8:27:14<00:00,  2.98s/it]   


In [5]:
all_user_ref = {}
with open('/Users/fedor/Desktop/meta_project/data/metacritic/url/critic_refs/final_user_refs.json') as f:
    all_user_ref = json.load(f)
len(all_user_ref)

12927